ASAP CRN Metadata validation - wave 2

# Team Sulzer. ASAP CRN Metadata validation - wave 2

13 Oct 2024
Andy Henrie




In [1]:
import pandas as pd
from pathlib import Path
import os, sys

sys.path.append(os.path.abspath((os.path.join(os.getcwd(), 'src/crn_utils'))))

from util import read_CDE, NULL, prep_table, read_meta_table, create_metadata_package
from validate import validate_table, ReportCollector
from update_schema import v1_to_v2, v2_to_v3_PMDBS, intervention_typer
from checksums import extract_md5_from_details2, get_md5_hashes 
from bucket_util import authenticate_with_service_account, gsutil_ls, gsutil_cp, gsutil_mv 

%load_ext autoreload
%autoreload 2

root_path = Path.home() / ("Projects/ASAP/data/teams")


Streamlit NOT successfully imported


## CDEs
load the relavent CDEs

In [2]:
schema_version = "v1"
schema_path = Path.home() / "Projects/ASAP/crn-utils/resource/CDE"
CDEv1 = read_CDE(schema_version, local_path=schema_path)
schema_version = "v2.1"
CDEv2 = read_CDE(schema_version, local_path=schema_path)
schema_version = "v3.0"
CDEv3 = read_CDE(schema_version, local_path=schema_path)

metadata_version: ASAP_CDE_v1
https://docs.google.com/spreadsheets/d/1c0z5KvRELdT2AtQAH2Dus8kwAyyLrR0CROhKOjpU4Vc/gviz/tq?tqx=out:csv&sheet=v1
/Users/ergonyc/Projects/ASAP/crn-utils/resource/CDE/ASAP_CDE_v1.csv
read local file
metadata_version: ASAP_CDE_v2.1
https://docs.google.com/spreadsheets/d/1c0z5KvRELdT2AtQAH2Dus8kwAyyLrR0CROhKOjpU4Vc/gviz/tq?tqx=out:csv&sheet=v2.1
/Users/ergonyc/Projects/ASAP/crn-utils/resource/CDE/ASAP_CDE_v2.1.csv
read local file
metadata_version: ASAP_CDE_v3.0
https://docs.google.com/spreadsheets/d/1c0z5KvRELdT2AtQAH2Dus8kwAyyLrR0CROhKOjpU4Vc/gviz/tq?tqx=out:csv&sheet=v3.0
/Users/ergonyc/Projects/ASAP/crn-utils/resource/CDE/ASAP_CDE_v3.0.csv
read local file


## Load original tables 
These were submitted as v2.1

### Team Sulzer

In [3]:
## convert 
team = "sulzer"
dataset_name = "sn-rnaseq"

metadata_path = root_path / f"{team}/{dataset_name}/metadata"
og_path = metadata_path / "og"


metadata_version = "v2"

METADATA_VERSION_DATE = f"{metadata_version}_{pd.Timestamp.now().strftime('%Y%m%d')}"

In [4]:
CDE = CDEv2
tables = CDE['Table'].unique()

dfs = {}
for table in tables:
    df = read_meta_table(og_path / f"{table}.csv")
    schema = CDE[CDE['Table'] == table]
    report = ReportCollector(destination="NA")
    full_table, report = validate_table(df.copy(), table, schema, report)
    report.print_log()
    dfs[table] = full_table
    # df.to_csv(v1_path / f"{table}.csv", index=False)

recoding number_of_brain_samples as int
🚨⚠️❗ **Missing Required Fields in STUDY: submitter_email**
🚨⚠️❗ **Missing Optional Fields in STUDY: alternate_dataset_id**
🚨⚠️❗ **8 Fields with empty (NULL) values:**

	- ASAP_grant_id: 1/1 empty rows (REQUIRED)

	- other_funding_source: 1/1 empty rows (REQUIRED)

	- publication_DOI: 1/1 empty rows (REQUIRED)

	- publication_PMID: 1/1 empty rows (REQUIRED)

	- PI_ORCHID: 1/1 empty rows (OPTIONAL)

	- PI_google_scholar_id: 1/1 empty rows (OPTIONAL)

	- preprocessing_references: 1/1 empty rows (OPTIONAL)

	- metadata_version_date: 1/1 empty rows (OPTIONAL)
No invalid entries found in Enum fields.
🚨⚠️❗ **Extra field in STUDY: submittor_email**

All required fields are present in *PROTOCOL* table.
🚨⚠️❗ **3 Fields with empty (NULL) values:**

	- github_url: 1/1 empty rows (REQUIRED)

	- protocols_io_DOI: 1/1 empty rows (REQUIRED)

	- other_reference: 1/1 empty rows (OPTIONAL)
No invalid entries found in Enum fields.

recoding age_at_onset as int
recod

In [5]:

SUBJECT = read_meta_table(og_path / "SUBJECT.csv")
CLINPATH = read_meta_table(og_path / "CLINPATH.csv")
STUDY = read_meta_table(og_path / "STUDY.csv")
PROTOCOL = read_meta_table(og_path / "PROTOCOL.csv")
SAMPLE = read_meta_table(og_path / "SAMPLE.csv")
DATA = read_meta_table(og_path / "DATA.csv")



In [6]:
STUDY['metadata_version_date'] = METADATA_VERSION_DATE
STUDY.rename(columns={"submittor_email":"submitter_email"}, inplace=True)
STUDY['alternate_dataset_id'] = NULL
STUDY['team_dataset_id'] = dataset_name.replace(" ", "_").replace("-", "_")


In [7]:

SUBJECT = prep_table(SUBJECT, CDE)
SUBJECT['sex'] = SUBJECT['sex'].replace({"M":"Male", "F":"Female"})
SUBJECT.head()


,subject_id,source_subject_id,AMPPD_id,GP2_id,biobank_name,organism,sex,age_at_collection,race,ethnicity,...,hx_dementia_mci,hx_melanoma,education_level,smoking_status,smoking_years,APOE_e4_status,cognitive_status,time_from_baseline,primary_diagnosis,primary_diagnosis_text
0,OJ9,T-4530,NA,NA,New York Brain Bank,Human,Male,77,Unknown,Unknown,...,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown
1,OJ8,T-4105,NA,NA,New York Brain Bank,Human,Female,89+,Unknown,Hispanic or Latino,...,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown
2,OJ7,T-5259,NA,NA,New York Brain Bank,Human,Male,79,White,Not Hispanic or Latino,...,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown
3,OJ6,T-5385,NA,NA,New York Brain Bank,Human,Male,88,White,Not Hispanic or Latino,...,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown
4,OJ5,T-2465,NA,NA,New York Brain Bank,Human,Female,83,Black or African American,Not Hispanic or Latino,...,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown


In [8]:
SUBJECT['subject_id'].shape


(46,)

In [9]:
SUBJECT['source_subject_id'].unique()


array(['T-4530', 'T-4105', 'T-5259', 'T-5385', 'T-2465', 'T-3799',
       'T-4424', 'T-5133', 'T-5227', 'T-5547', 'T-5359', 'T-4116',
       'T-4839', 'T-2007', 'T-4938', 'T-5498', 'T-2791', 'T-4695',
       'T-4934', 'T-3870', 'T-124', 'T-81', 'T-4515', 'T-4235', 'T-4815',
       'T-4857', 'T-3998', 'T-4462', 'T-4108', 'T-4174', 'T3860',
       'T-4162', 'T-4210', 'T-233', 'T1976', 'T-2017', 'T-308', 'T-1952',
       'T-777', 'T-1619', 'T290', 'T-551', 'T-257', 'T-343', 'T-362'],
      dtype=object)

In [10]:


SUBJECT['hx_dementia_mci'] = SUBJECT['hx_dementia_mci'].replace("Unknown", NULL)

SUBJECT['hx_melanoma'] = SUBJECT['hx_melanoma'].replace("Unknown", NULL)
SUBJECT['education_level'] = SUBJECT['education_level'].replace("Unknown", NULL)
SUBJECT['cognitive_status'] = SUBJECT['cognitive_status'].replace("Unknown", NULL)
SUBJECT['primary_diagnosis'] = SUBJECT['primary_diagnosis'].replace("Unknown", NULL)


In [11]:
SAMPLE = prep_table(SAMPLE, CDE)

SAMPLE["organism_ontology_term_id"] = "NCBITaxon:9606"
SAMPLE['molecular_source'] = SAMPLE['molecular_source'].replace("RNA", "PolyA RNA")
SAMPLE['region_level_2'] = SAMPLE['region_level_2'].replace('Substantia Nigra', 'Substantia nigra',)
SAMPLE.head()

,sample_id,subject_id,source_sample_id,replicate,replicate_count,repeated_sample,batch,tissue,brain_region,hemisphere,...,sex_ontology_term_id,self_reported_ethnicity_ontology_term_id,disease_ontology_term_id,tissue_ontology_term_id,cell_type_ontology_term_id,assay_ontology_term_id,suspension_type,DV200,pm_PH,donor_id
0,T-362,OF001,T-362,Rep1,1,0,1,Brain,Cingulate Cortex,NA,...,PATO:0000383 (female),Unknown,NA,UBERON:0003027,NA,NA,Nucleus,NA,NA,NA
1,T-308,OF009,T-308,Rep1,1,0,1,Brain,Cingulate Cortex,NA,...,PATO:0000384 (male),Unknown,NA,UBERON:0003027,NA,NA,Nucleus,NA,NA,NA
2,T3860,OF016,T3860,Rep1,1,0,1,Brain,Cingulate Cortex,NA,...,PATO:0000384 (male),Unknown,NA,UBERON:0003027,NA,NA,Nucleus,NA,NA,NA
3,T290,OF005,T290,Rep1,1,0,1,Brain,Cingulate Cortex,NA,...,PATO:0000384 (male),Unknown,NA,UBERON:0003027,NA,NA,Nucleus,NA,NA,NA
4,T-233,OF013,T-233,Rep1,1,0,1,Brain,Cingulate Cortex,NA,...,PATO:0000384 (male),Unknown,NA,UBERON:0003027,NA,NA,Nucleus,NA,NA,NA


In [12]:
fix_samples = {'T3860':'T-3860', 'T290':'T-290','T1976':'T-1976'}
SAMPLE['sample_id'] = SAMPLE['sample_id'].replace(fix_samples)
SAMPLE['source_sample_id'] = SAMPLE['source_sample_id'].replace(fix_samples)
SAMPLE.head()

,sample_id,subject_id,source_sample_id,replicate,replicate_count,repeated_sample,batch,tissue,brain_region,hemisphere,...,sex_ontology_term_id,self_reported_ethnicity_ontology_term_id,disease_ontology_term_id,tissue_ontology_term_id,cell_type_ontology_term_id,assay_ontology_term_id,suspension_type,DV200,pm_PH,donor_id
0,T-362,OF001,T-362,Rep1,1,0,1,Brain,Cingulate Cortex,NA,...,PATO:0000383 (female),Unknown,NA,UBERON:0003027,NA,NA,Nucleus,NA,NA,NA
1,T-308,OF009,T-308,Rep1,1,0,1,Brain,Cingulate Cortex,NA,...,PATO:0000384 (male),Unknown,NA,UBERON:0003027,NA,NA,Nucleus,NA,NA,NA
2,T-3860,OF016,T-3860,Rep1,1,0,1,Brain,Cingulate Cortex,NA,...,PATO:0000384 (male),Unknown,NA,UBERON:0003027,NA,NA,Nucleus,NA,NA,NA
3,T-290,OF005,T-290,Rep1,1,0,1,Brain,Cingulate Cortex,NA,...,PATO:0000384 (male),Unknown,NA,UBERON:0003027,NA,NA,Nucleus,NA,NA,NA
4,T-233,OF013,T-233,Rep1,1,0,1,Brain,Cingulate Cortex,NA,...,PATO:0000384 (male),Unknown,NA,UBERON:0003027,NA,NA,Nucleus,NA,NA,NA


In [13]:
batch_mapper = {"Cingulate Cortex":"cing1", "Substantia Nigra": "sn1"}
SAMPLE['batch']=SAMPLE['brain_region'].map(batch_mapper)

## 🚨⚠️❗ EXTREME CURATION HERE 🚨⚠️❗
The codeing by Team-Sulzer is inconsistent.  However I am able to infer the following:

TODO:  Define `sample_id`s from `subject_id`, and create `sample_id` as `source_subject_id`


`source_subject_id`s seem to be unique and correct.  Although the CLINPATH.duration_pmi are different for `source_subject_id`="T-2465", and some demographic info is missing for the cingulate sample, these seem to be two samples from the same subject.  

Hence:  We will assume `source_subject_id` is ground truth.   Redefine `sample_id` from the current `subject_id` (which matches file naming conventsions), and  create a _new_ `subject_id` from the `source_subject_id` by prepending "SUL_".  Finally, `source_sample_id` will be set to pd.NA as is currently just a copy of `source_subject_id`

### fix SAMPLE table first

In [14]:
# simple copy subject_id to sample_id, sample_id == source_sample_id
SAMPLE['sample_id'] = SAMPLE['subject_id']


In [15]:
fix_samples = {'T3860':'T-3860', 'T290':'T-290','T1976':'T-1976'}
SAMPLE['source_sample_id'] = SAMPLE['source_sample_id'].replace(fix_samples)
# then make a subject_id by copying souce_sample_id
SAMPLE['subject_id'] = "SUL_" + SAMPLE['source_sample_id'].str.replace("-","_")

# # finally set source_sample_id to pd.NA
SAMPLE['source_sample_id'] = NULL

SAMPLE['alternate_sample_id'] = NULL

SAMPLE.head()

,sample_id,subject_id,source_sample_id,replicate,replicate_count,repeated_sample,batch,tissue,brain_region,hemisphere,...,self_reported_ethnicity_ontology_term_id,disease_ontology_term_id,tissue_ontology_term_id,cell_type_ontology_term_id,assay_ontology_term_id,suspension_type,DV200,pm_PH,donor_id,alternate_sample_id
0,OF001,SUL_T_362,NA,Rep1,1,0,cing1,Brain,Cingulate Cortex,NA,...,Unknown,NA,UBERON:0003027,NA,NA,Nucleus,NA,NA,NA,NA
1,OF009,SUL_T_308,NA,Rep1,1,0,cing1,Brain,Cingulate Cortex,NA,...,Unknown,NA,UBERON:0003027,NA,NA,Nucleus,NA,NA,NA,NA
2,OF016,SUL_T_3860,NA,Rep1,1,0,cing1,Brain,Cingulate Cortex,NA,...,Unknown,NA,UBERON:0003027,NA,NA,Nucleus,NA,NA,NA,NA
3,OF005,SUL_T_290,NA,Rep1,1,0,cing1,Brain,Cingulate Cortex,NA,...,Unknown,NA,UBERON:0003027,NA,NA,Nucleus,NA,NA,NA,NA
4,OF013,SUL_T_233,NA,Rep1,1,0,cing1,Brain,Cingulate Cortex,NA,...,Unknown,NA,UBERON:0003027,NA,NA,Nucleus,NA,NA,NA,NA


### now SUBJECT table 

In [16]:
fix_samples = {'T3860':'T-3860', 'T290':'T-290','T1976':'T-1976'}
SUBJECT['source_subject_id'] = SUBJECT['source_subject_id'].replace(fix_samples)

SUBJECT['subject_id'] = "SUL_" + SUBJECT['source_subject_id'].str.replace("-","_")
SUBJECT['subject_id'].unique()

array(['SUL_T_4530', 'SUL_T_4105', 'SUL_T_5259', 'SUL_T_5385',
       'SUL_T_2465', 'SUL_T_3799', 'SUL_T_4424', 'SUL_T_5133',
       'SUL_T_5227', 'SUL_T_5547', 'SUL_T_5359', 'SUL_T_4116',
       'SUL_T_4839', 'SUL_T_2007', 'SUL_T_4938', 'SUL_T_5498',
       'SUL_T_2791', 'SUL_T_4695', 'SUL_T_4934', 'SUL_T_3870',
       'SUL_T_124', 'SUL_T_81', 'SUL_T_4515', 'SUL_T_4235', 'SUL_T_4815',
       'SUL_T_4857', 'SUL_T_3998', 'SUL_T_4462', 'SUL_T_4108',
       'SUL_T_4174', 'SUL_T_3860', 'SUL_T_4162', 'SUL_T_4210',
       'SUL_T_233', 'SUL_T_1976', 'SUL_T_2017', 'SUL_T_308', 'SUL_T_1952',
       'SUL_T_777', 'SUL_T_1619', 'SUL_T_290', 'SUL_T_551', 'SUL_T_257',
       'SUL_T_343', 'SUL_T_362'], dtype=object)

### and CLINPATH table is already coded as it should be



In [17]:

fix_samples = {'T3860':'T-3860', 'T290':'T-290','T1976':'T-1976'}
CLINPATH['source_subject_id'] = CLINPATH['source_subject_id'].replace(fix_samples)
CLINPATH['source_subject_id'].unique()

array(['T-4530', 'T-4105', 'T-5259', 'T-5385', 'T-2465', 'T-3799',
       'T-4424', 'T-5133', 'T-5227', 'T-5547', 'T-5359', 'T-4116',
       'T-4839', 'T-2007', 'T-4938', 'T-5498', 'T-2791', 'T-4695',
       'T-4934', 'T-3870', 'T-124', 'T-81', 'T-4515', 'T-4235', 'T-4815',
       'T-4857', 'T-3998', 'T-4462', 'T-4108', 'T-4174', 'T-3860',
       'T-4162', 'T-4210', 'T-233', 'T-1976', 'T-2017', 'T-308', 'T-1952',
       'T-777', 'T-1619', 'T-290', 'T-551', 'T-257', 'T-343', 'T-362'],
      dtype=object)

In [18]:
CLINPATH['subject_id'] = "SUL_" + CLINPATH['source_subject_id'].str.replace("-","_")
CLINPATH.head()

,subject_id,source_subject_id,duration_pmi,path_autopsy_dx_main,path_autopsy_second_dx,path_autopsy_third_dx,path_autopsy_fourth_dx,path_autopsy_fifth_dx,path_autopsy_sixth_dx,path_autopsy_seventh_dx,...,path_nia_ri,path_nia_aa_a,path_nia_aa_b,path_nia_aa_c,TDP43,arteriolosclerosis_severity_scale,amyloid_angiopathy_severity_scale,path_ad_level,dig_slide_avail,quant_path_avail
0,SUL_T_4530,T-4530,17.06666667,Parkinson's disease with dementia,NA,NA,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,No,No
1,SUL_T_4105,T-4105,20,Alzheimer's disease (intermediate level neurop...,NA,NA,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,No,No
2,SUL_T_5259,T-5259,10.5,Parkinson's disease with dementia,NA,NA,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,No,No
3,SUL_T_5385,T-5385,14.05,"Control, Primary age-related tauopathy (PART)",NA,NA,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,No,No
4,SUL_T_2465,T-2465,9.25,"Control, Low level AD neuropathological change",NA,NA,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,No,No


## DATA table

The DATA.csv was not filled out.  So we will construct it by joining the manifest from the source of data transfer for Team Sulzer.

### read_file_list
Helper function to read sulzer manifest 

In [19]:

cingulate_files = Path.home() / "Projects/ASAP/team-sulzer/cingulate_manifest.txt"
nigra_files = Path.home() / "Projects/ASAP/team-sulzer/nigra_manifest.txt"

In [20]:
# tmp2 = pd.read_csv("sulzer_fastqs.txt", delimiter=" ", header=None, names=['file_name'])
def read_gz_file_list(file_nm):
    # Read the text file
    with open(file_nm, 'r') as file:
        lines = [line.strip() for line in file]

    # Create a DataFrame
    df = pd.DataFrame(lines, columns=['file_name'])

    df['parts'] = df['file_name'].apply(lambda x: x.lstrip("gs://").split("/"))
    df['source'] = df['parts'].apply(lambda x: x[1].lstrip("fastqs_") )
    df['name'] = df['parts'].apply(lambda x: x[-1])

    df['name_parts'] = df['name'].apply(lambda x: x.rstrip('.fastq.gz').split("_"))
    df['sample_id'] = df['name_parts'].apply(lambda x: x[0])
    df['lane_id'] = df['name_parts'].apply(lambda x: x[2])
    df['file_type'] = df['name_parts'].apply(lambda x: x[3])
    df.drop(columns=['parts','name_parts'], inplace=True)
    return df

# tmp2 = pd.read_csv("sulzer_fastqs.txt", delimiter=" ", header=None, names=['file_name'])
def read_file_list_sulzer(file_nm):
    # Read the text file
    keep_ends = ("fastq.gz", "bam", "tar")
    with open(file_nm, 'r') as file:
        lines = [line.strip() for line in file if line.strip().endswith(keep_ends)]

    # Create a DataFrame
    df = pd.DataFrame(lines, columns=['file_name'])

    df['parts'] = df['file_name'].apply(lambda x: x.split("/"))
    # df['source'] = df['parts'].apply(lambda x: x[5].lstrip("fastqs_") )
    df['file_name'] = df['parts'].apply(lambda x: x[-1])
    df['sample_id'] = df['parts'].apply(lambda x: x[6] if x[-1].endswith(("fastq.gz","tar")) else x[7])
    df['file_type'] = df['parts'].apply(lambda x: x[7] if x[7].strip() in ["fastq","analysis"] else "bam")
    # df['tmp1'] = df['parts'].apply(lambda x: x[7] if len(x) > 7 else pd.NA)

    # df['name_parts'] = df['name'].apply(lambda x: x.rstrip('.fastq.gz').split("_"))
    # df['sample_id'] = df['name_parts'].apply(lambda x: x[0])
    # df['lane_id'] = df['name_parts'].apply(lambda x: x[2])
    # df['file_type'] = df['name_parts'].apply(lambda x: x[3])
    df.drop(columns=['parts'], inplace=True)
    return df




In [21]:

c_fastqs = read_file_list_sulzer(cingulate_files)
c_fastqs['src'] = "cingulate"
n_fastqs = read_file_list_sulzer(nigra_files)
n_fastqs['src'] = "nigra"

from_source = pd.concat([c_fastqs, n_fastqs])

In [22]:
n_fastqs

,file_name,sample_id,file_type,src
0,OJ1_cellranger_count_outs.tar,OJ1,analysis,nigra
1,OJ1_S1_L001_I1_001.fastq.gz,OJ1,fastq,nigra
2,OJ1_S1_L001_I2_001.fastq.gz,OJ1,fastq,nigra
3,OJ1_S1_L001_R1_001.fastq.gz,OJ1,fastq,nigra
4,OJ1_S1_L001_R2_001.fastq.gz,OJ1,fastq,nigra
...,...,...,...,...
454,OJ9_S1_L003_R2_001.fastq.gz,OJ9,fastq,nigra
455,OJ9_S1_L004_I1_001.fastq.gz,OJ9,fastq,nigra
456,OJ9_S1_L004_I2_001.fastq.gz,OJ9,fastq,nigra
457,OJ9_S1_L004_R1_001.fastq.gz,OJ9,fastq,nigra


In [23]:
## now make DATA from scratch to ready to join into
# Start with sample_id
# DATA_ = read_meta_table(metadata_path / "DATA.csv")

DATA = SAMPLE[["sample_id", "replicate", "replicate_count","repeated_sample","batch"]].copy()

DATA.head()



,sample_id,replicate,replicate_count,repeated_sample,batch
0,OF001,Rep1,1,0,cing1
1,OF009,Rep1,1,0,cing1
2,OF016,Rep1,1,0,cing1
3,OF005,Rep1,1,0,cing1
4,OF013,Rep1,1,0,cing1


In [24]:
DATA = pd.merge(DATA, from_source, on='sample_id', how='left')
DATA.tail()

,sample_id,replicate,replicate_count,repeated_sample,batch,file_name,file_type,src
707,OJ26,Rep1,1,0,sn1,OJ26_S1_L003_R2_001.fastq.gz,fastq,nigra
708,OJ26,Rep1,1,0,sn1,OJ26_S1_L004_I1_001.fastq.gz,fastq,nigra
709,OJ26,Rep1,1,0,sn1,OJ26_S1_L004_I2_001.fastq.gz,fastq,nigra
710,OJ26,Rep1,1,0,sn1,OJ26_S1_L004_R1_001.fastq.gz,fastq,nigra
711,OJ26,Rep1,1,0,sn1,OJ26_S1_L004_R2_001.fastq.gz,fastq,nigra


In [25]:
DATA['omic'] = 'RNA'
DATA['adjustments'] = 'Raw'
DATA['technology'] = "SN"
DATA['file_description']= DATA['file_type'].map({"bam": "Binary aligned sequence",
                                                "fastq": "gzip fastq",
                                                "analysis": "cellranger counts tar archive"})
DATA['file_description']= DATA['file_type'].map({"bam": "Reads",
                                                "fastq": "Reads",
                                                "analysis": "Counts"})

# now fix the file_type
DATA['file_type'] = DATA['file_type'].replace({'fastq':'fastq', "bam":'Per sample raw file', "analysis":'Per sample processed file'})
DATA['content'] = 'Counts'



get "md5s"

In [26]:


cingulate_md5s = Path.home() / "Projects/ASAP/team-sulzer/cingulate_fastq_md5.log"
nigra_md5s = Path.home() / "Projects/ASAP/team-sulzer/nigra_fastq_md5.log"

In [27]:
from checksums import extract_hashes_from_gsutil

# cingulate
cingulate_hashs = extract_hashes_from_gsutil(cingulate_md5s)

# nigra
nigra_hashs = extract_hashes_from_gsutil(nigra_md5s)

# combine cingulate and nigra
source_file_crc = cingulate_hashs[0] | nigra_hashs[0]
source_file_md5 = cingulate_hashs[1] | nigra_hashs[1]


In [28]:
DATA['file_MD5'] = DATA['file_name'].map(source_file_md5)

In [29]:

DATA.head()



,sample_id,replicate,replicate_count,repeated_sample,batch,file_name,file_type,src,omic,adjustments,technology,file_description,content,file_MD5
0,OF001,Rep1,1,0,cing1,possorted_genome_bam.bam,Per sample raw file,cingulate,RNA,Raw,SN,Reads,Counts,NaN
1,OF001,Rep1,1,0,cing1,OF001_cellranger_count_outs.tar,Per sample processed file,cingulate,RNA,Raw,SN,Counts,Counts,NaN
2,OF001,Rep1,1,0,cing1,OF001_S1_L001_I1_001.fastq.gz,fastq,cingulate,RNA,Raw,SN,Reads,Counts,74a74c8929b01a6da7e79b83f91a6607
3,OF001,Rep1,1,0,cing1,OF001_S1_L001_I2_001.fastq.gz,fastq,cingulate,RNA,Raw,SN,Reads,Counts,644d95728b50f1ff9db7becfa5b5c394
4,OF001,Rep1,1,0,cing1,OF001_S1_L001_R1_001.fastq.gz,fastq,cingulate,RNA,Raw,SN,Reads,Counts,129dfb6fe6115966e2cf2af5e3c82bdf


# No data found for `OJ8`!!!

# load manifests and md5s

Separate manifests for cingulate and nigra to make it easier for Team Sulzer to collect.


In [30]:
SAMPLE[SAMPLE['sample_id']=='OJ8']
empty_sample = SAMPLE['sample_id']=='OJ8'
print(SAMPLE.shape)
SAMPLE = SAMPLE[~empty_sample]
SAMPLE.shape

(46, 34)


(45, 34)

In [31]:
DATA[DATA['sample_id']=='OJ8']
empty_sample = DATA['sample_id']=='OJ8'
print(DATA.shape)
DATA = DATA[~empty_sample]
DATA.shape

(712, 14)


(711, 14)

In [32]:
DATA.rename(columns={'adjustments':'adjustment'}, inplace=True)
DATA['time'] = NULL
DATA['header'] = NULL
DATA['annotation'] = NULL
DATA['configuration_file'] = NULL

DATA.head()

,sample_id,replicate,replicate_count,repeated_sample,batch,file_name,file_type,src,omic,adjustment,technology,file_description,content,file_MD5,time,header,annotation,configuration_file
0,OF001,Rep1,1,0,cing1,possorted_genome_bam.bam,Per sample raw file,cingulate,RNA,Raw,SN,Reads,Counts,NaN,NA,NA,NA,NA
1,OF001,Rep1,1,0,cing1,OF001_cellranger_count_outs.tar,Per sample processed file,cingulate,RNA,Raw,SN,Counts,Counts,NaN,NA,NA,NA,NA
2,OF001,Rep1,1,0,cing1,OF001_S1_L001_I1_001.fastq.gz,fastq,cingulate,RNA,Raw,SN,Reads,Counts,74a74c8929b01a6da7e79b83f91a6607,NA,NA,NA,NA
3,OF001,Rep1,1,0,cing1,OF001_S1_L001_I2_001.fastq.gz,fastq,cingulate,RNA,Raw,SN,Reads,Counts,644d95728b50f1ff9db7becfa5b5c394,NA,NA,NA,NA
4,OF001,Rep1,1,0,cing1,OF001_S1_L001_R1_001.fastq.gz,fastq,cingulate,RNA,Raw,SN,Reads,Counts,129dfb6fe6115966e2cf2af5e3c82bdf,NA,NA,NA,NA


In [33]:
SUBJECT['subject_id'].unique()

array(['SUL_T_4530', 'SUL_T_4105', 'SUL_T_5259', 'SUL_T_5385',
       'SUL_T_2465', 'SUL_T_3799', 'SUL_T_4424', 'SUL_T_5133',
       'SUL_T_5227', 'SUL_T_5547', 'SUL_T_5359', 'SUL_T_4116',
       'SUL_T_4839', 'SUL_T_2007', 'SUL_T_4938', 'SUL_T_5498',
       'SUL_T_2791', 'SUL_T_4695', 'SUL_T_4934', 'SUL_T_3870',
       'SUL_T_124', 'SUL_T_81', 'SUL_T_4515', 'SUL_T_4235', 'SUL_T_4815',
       'SUL_T_4857', 'SUL_T_3998', 'SUL_T_4462', 'SUL_T_4108',
       'SUL_T_4174', 'SUL_T_3860', 'SUL_T_4162', 'SUL_T_4210',
       'SUL_T_233', 'SUL_T_1976', 'SUL_T_2017', 'SUL_T_308', 'SUL_T_1952',
       'SUL_T_777', 'SUL_T_1619', 'SUL_T_290', 'SUL_T_551', 'SUL_T_257',
       'SUL_T_343', 'SUL_T_362'], dtype=object)

In [34]:
CLINPATH['subject_id'].unique()

array(['SUL_T_4530', 'SUL_T_4105', 'SUL_T_5259', 'SUL_T_5385',
       'SUL_T_2465', 'SUL_T_3799', 'SUL_T_4424', 'SUL_T_5133',
       'SUL_T_5227', 'SUL_T_5547', 'SUL_T_5359', 'SUL_T_4116',
       'SUL_T_4839', 'SUL_T_2007', 'SUL_T_4938', 'SUL_T_5498',
       'SUL_T_2791', 'SUL_T_4695', 'SUL_T_4934', 'SUL_T_3870',
       'SUL_T_124', 'SUL_T_81', 'SUL_T_4515', 'SUL_T_4235', 'SUL_T_4815',
       'SUL_T_4857', 'SUL_T_3998', 'SUL_T_4462', 'SUL_T_4108',
       'SUL_T_4174', 'SUL_T_3860', 'SUL_T_4162', 'SUL_T_4210',
       'SUL_T_233', 'SUL_T_1976', 'SUL_T_2017', 'SUL_T_308', 'SUL_T_1952',
       'SUL_T_777', 'SUL_T_1619', 'SUL_T_290', 'SUL_T_551', 'SUL_T_257',
       'SUL_T_343', 'SUL_T_362'], dtype=object)

In [35]:
SUBJECT['subject_id'].unique()
# CLINPATH.head()

array(['SUL_T_4530', 'SUL_T_4105', 'SUL_T_5259', 'SUL_T_5385',
       'SUL_T_2465', 'SUL_T_3799', 'SUL_T_4424', 'SUL_T_5133',
       'SUL_T_5227', 'SUL_T_5547', 'SUL_T_5359', 'SUL_T_4116',
       'SUL_T_4839', 'SUL_T_2007', 'SUL_T_4938', 'SUL_T_5498',
       'SUL_T_2791', 'SUL_T_4695', 'SUL_T_4934', 'SUL_T_3870',
       'SUL_T_124', 'SUL_T_81', 'SUL_T_4515', 'SUL_T_4235', 'SUL_T_4815',
       'SUL_T_4857', 'SUL_T_3998', 'SUL_T_4462', 'SUL_T_4108',
       'SUL_T_4174', 'SUL_T_3860', 'SUL_T_4162', 'SUL_T_4210',
       'SUL_T_233', 'SUL_T_1976', 'SUL_T_2017', 'SUL_T_308', 'SUL_T_1952',
       'SUL_T_777', 'SUL_T_1619', 'SUL_T_290', 'SUL_T_551', 'SUL_T_257',
       'SUL_T_343', 'SUL_T_362'], dtype=object)

In [36]:
SAMPLE

,sample_id,subject_id,source_sample_id,replicate,replicate_count,repeated_sample,batch,tissue,brain_region,hemisphere,...,self_reported_ethnicity_ontology_term_id,disease_ontology_term_id,tissue_ontology_term_id,cell_type_ontology_term_id,assay_ontology_term_id,suspension_type,DV200,pm_PH,donor_id,alternate_sample_id
0,OF001,SUL_T_362,NA,Rep1,1,0,cing1,Brain,Cingulate Cortex,NA,...,Unknown,NA,UBERON:0003027,NA,NA,Nucleus,NA,NA,NA,NA
1,OF009,SUL_T_308,NA,Rep1,1,0,cing1,Brain,Cingulate Cortex,NA,...,Unknown,NA,UBERON:0003027,NA,NA,Nucleus,NA,NA,NA,NA
2,OF016,SUL_T_3860,NA,Rep1,1,0,cing1,Brain,Cingulate Cortex,NA,...,Unknown,NA,UBERON:0003027,NA,NA,Nucleus,NA,NA,NA,NA
3,OF005,SUL_T_290,NA,Rep1,1,0,cing1,Brain,Cingulate Cortex,NA,...,Unknown,NA,UBERON:0003027,NA,NA,Nucleus,NA,NA,NA,NA
4,OF013,SUL_T_233,NA,Rep1,1,0,cing1,Brain,Cingulate Cortex,NA,...,Unknown,NA,UBERON:0003027,NA,NA,Nucleus,NA,NA,NA,NA
5,OF018,SUL_T_4108,NA,Rep1,1,0,cing1,Brain,Cingulate Cortex,NA,...,Unknown,NA,UBERON:0003027,NA,NA,Nucleus,NA,NA,NA,NA
6,OF003,SUL_T_257,NA,Rep1,1,0,cing1,Brain,Cingulate Cortex,NA,...,Unknown,NA,UBERON:0003027,NA,NA,Nucleus,NA,NA,NA,NA
7,OF007,SUL_T_777,NA,Rep1,1,0,cing1,Brain,Cingulate Cortex,NA,...,Unknown,NA,UBERON:0003027,NA,NA,Nucleus,NA,NA,NA,NA
8,OF011,SUL_T_1976,NA,Rep1,1,0,cing1,Brain,Cingulate Cortex,NA,...,Unknown,NA,UBERON:0003027,NA,NA,Nucleus,NA,NA,NA,NA
9,OF015,SUL_T_4162,NA,Rep1,1,0,cing1,Brain,Cingulate Cortex,NA,...,Unknown,NA,UBERON:0003027,NA,NA,Nucleus,NA,NA,NA,NA


In [37]:
dict(zip(CLINPATH['subject_id'], CLINPATH['path_autopsy_dx_main']))

{'SUL_T_4530': "Parkinson's disease with dementia",
 'SUL_T_4105': "Alzheimer's disease (intermediate level neuropathological change)",
 'SUL_T_5259': "Parkinson's disease with dementia",
 'SUL_T_5385': 'Control, Primary age-related tauopathy (PART)',
 'SUL_T_2465': 'Control, Low level AD neuropathological change',
 'SUL_T_3799': 'Other neurological disorder',
 'SUL_T_4424': "Parkinson's disease with dementia",
 'SUL_T_5133': "Alzheimer's disease (intermediate level neuropathological change)",
 'SUL_T_5227': "Alzheimer's disease (intermediate level neuropathological change)",
 'SUL_T_5547': 'Control, Primary age-related tauopathy (PART)',
 'SUL_T_5359': "Alzheimer's disease (intermediate level neuropathological change)",
 'SUL_T_4116': "Alzheimer's disease (intermediate level neuropathological change)",
 'SUL_T_4839': "Parkinson's disease with dementia",
 'SUL_T_2007': "Parkinson's disease with dementia",
 'SUL_T_4938': "Parkinson's disease with dementia",
 'SUL_T_5498': "Parkinson's d

In [38]:
SUBJECT['primary_diagnosis'] = SUBJECT['subject_id'].map(dict(zip(CLINPATH['subject_id'], CLINPATH['path_autopsy_dx_main'])))
SUBJECT['primary_diagnosis']

0                     Parkinson's disease with dementia
1     Alzheimer's disease (intermediate level neurop...
2                     Parkinson's disease with dementia
3         Control, Primary age-related tauopathy (PART)
4        Control, Low level AD neuropathological change
5                           Other neurological disorder
6                     Parkinson's disease with dementia
7     Alzheimer's disease (intermediate level neurop...
8     Alzheimer's disease (intermediate level neurop...
9         Control, Primary age-related tauopathy (PART)
10    Alzheimer's disease (intermediate level neurop...
11    Alzheimer's disease (intermediate level neurop...
12                    Parkinson's disease with dementia
13                    Parkinson's disease with dementia
14                    Parkinson's disease with dementia
15                    Parkinson's disease with dementia
16       Control, Low level AD neuropathological change
17                    Parkinson's disease with d

In [39]:
# via chatGPT
diagnosis_mapping = {
    "Parkinson's disease with dementia": "Dementia with Lewy bodies",
    "Alzheimer's disease (intermediate level neuropathological change)": "Alzheimer's disease",
    "Control, Primary age-related tauopathy (PART)": "Other neurological disorder",
    "Control, Low level AD neuropathological change": "Prodromal non-motor PD",
    "Control, Limbic predominant age-related TDP43 proteinopathy (LATE)": "Frontotemporal dementia",
    "Parkinson's disease": "Idiopathic PD",
    "Control, no misfolded protein or significant vascular pathology": "No PD nor other neurological disorder"
}


SUBJECT['primary_diagnosis'] = SUBJECT['primary_diagnosis'].replace(diagnosis_mapping)

In [40]:
v2_path = metadata_path / "v2"
STUDY.to_csv(v2_path / f"STUDY.csv", index=False)
SAMPLE.to_csv(v2_path / f"SAMPLE.csv", index=False)
SUBJECT.to_csv(v2_path / f"SUBJECT.csv", index=False)
PROTOCOL.to_csv(v2_path / f"PROTOCOL.csv", index=False)
CLINPATH.to_csv(v2_path / f"CLINPATH.csv", index=False)
DATA.to_csv(v2_path / f"DATA.csv", index=False)

## validate v2 tables


In [41]:
CDE = CDEv2
tables = CDE['Table'].unique()
dfs = {}
for table in tables:
    schema = CDE[CDE['Table'] == table]
    df = read_meta_table(v2_path / f"{table}.csv")
    report = ReportCollector(destination="NA")
    full_table, report = validate_table(df.copy(), table, schema, report)
    dfs[table] = full_table
    report.print_log()

recoding number_of_brain_samples as int
All required fields are present in *STUDY* table.
🚨⚠️❗ **8 Fields with empty (NULL) values:**

	- ASAP_grant_id: 1/1 empty rows (REQUIRED)

	- other_funding_source: 1/1 empty rows (REQUIRED)

	- publication_DOI: 1/1 empty rows (REQUIRED)

	- publication_PMID: 1/1 empty rows (REQUIRED)

	- PI_ORCHID: 1/1 empty rows (OPTIONAL)

	- PI_google_scholar_id: 1/1 empty rows (OPTIONAL)

	- preprocessing_references: 1/1 empty rows (OPTIONAL)

	- alternate_dataset_id: 1/1 empty rows (OPTIONAL)
No invalid entries found in Enum fields.

All required fields are present in *PROTOCOL* table.
🚨⚠️❗ **3 Fields with empty (NULL) values:**

	- github_url: 1/1 empty rows (REQUIRED)

	- protocols_io_DOI: 1/1 empty rows (REQUIRED)

	- other_reference: 1/1 empty rows (OPTIONAL)
No invalid entries found in Enum fields.

recoding age_at_onset as int
recoding age_at_diagnosis as int
recoding first_motor_symptom as int
All required fields are present in *SUBJECT* table.
🚨⚠️❗ 

In [42]:
tables

array(['STUDY', 'PROTOCOL', 'SUBJECT', 'CLINPATH', 'SAMPLE', 'DATA'],
      dtype=object)

### save extras as auxillary tables


In [43]:
CDE = CDEv2
tables = CDE['Table'].unique()

dfs = {}
for table in tables:
    df = read_meta_table(v2_path / f"{table}.csv")
    schema = CDE[CDE['Table'] == table]

    valid_fields = schema['Field'].unique()
    df_out = df[valid_fields]
    aux_fields = set(df.columns) - set(valid_fields)
    if aux_fields:
        df_aux = df[list(aux_fields)]
        df_aux.to_csv(v2_path / f"{table}_auxiliary.csv", index=False)
        print(f"Saved {table}_auxiliary.csv")
    df_out.to_csv(v2_path / f"{table}.csv", index=False)
    dfs[table] = df_out

Saved DATA_auxiliary.csv


In [44]:
# make tables conform to CDE and save extra columns as "auxiliary"
v2_path = metadata_path / "v2"

for table in tables:
    df = dfs[table]
    schema = CDE[CDE['Table'] == table]
    valid_fields = schema['Field'].unique()
    df_out = df[valid_fields]
    aux_fields = set(df.columns) - set(valid_fields)
    if aux_fields:
        df_aux = df[list(aux_fields)]
        df_aux.to_csv(v2_path / f"{table}_auxiliary.csv", index=False)
        print(f"Saved {table}_auxiliary.csv")
    df_out.to_csv(v2_path / f"{table}.csv", index=False)

validate v2 tables


In [45]:
CDE = CDEv2
v2_tables = dfs
for table,df in v2_tables.items():
    schema = CDE[CDE['Table'] == table]

    report = ReportCollector(destination="NA")
    full_table, report = validate_table(df.copy(), table, schema, report)
    report.print_log()


recoding number_of_brain_samples as int
All required fields are present in *STUDY* table.
🚨⚠️❗ **8 Fields with empty (NULL) values:**

	- ASAP_grant_id: 1/1 empty rows (REQUIRED)

	- other_funding_source: 1/1 empty rows (REQUIRED)

	- publication_DOI: 1/1 empty rows (REQUIRED)

	- publication_PMID: 1/1 empty rows (REQUIRED)

	- PI_ORCHID: 1/1 empty rows (OPTIONAL)

	- PI_google_scholar_id: 1/1 empty rows (OPTIONAL)

	- preprocessing_references: 1/1 empty rows (OPTIONAL)

	- alternate_dataset_id: 1/1 empty rows (OPTIONAL)
No invalid entries found in Enum fields.

All required fields are present in *PROTOCOL* table.
🚨⚠️❗ **3 Fields with empty (NULL) values:**

	- github_url: 1/1 empty rows (REQUIRED)

	- protocols_io_DOI: 1/1 empty rows (REQUIRED)

	- other_reference: 1/1 empty rows (OPTIONAL)
No invalid entries found in Enum fields.

recoding age_at_onset as int
recoding age_at_diagnosis as int
recoding first_motor_symptom as int
All required fields are present in *SUBJECT* table.
🚨⚠️❗ 

In [46]:
v2_tables["SUBJECT"][['subject_id','primary_diagnosis'] ]
    # primary_diagnosis = v3_tables["SUBJECT"]['primary_diagnosis'].str.lower().str.replace(" ", "_")
    # diagnosis_mapper = dict(zip(subject_id, primary_diagnosis))
v2_tables["SUBJECT"]['subject_id'].unique()

array(['SUL_T_4530', 'SUL_T_4105', 'SUL_T_5259', 'SUL_T_5385',
       'SUL_T_2465', 'SUL_T_3799', 'SUL_T_4424', 'SUL_T_5133',
       'SUL_T_5227', 'SUL_T_5547', 'SUL_T_5359', 'SUL_T_4116',
       'SUL_T_4839', 'SUL_T_2007', 'SUL_T_4938', 'SUL_T_5498',
       'SUL_T_2791', 'SUL_T_4695', 'SUL_T_4934', 'SUL_T_3870',
       'SUL_T_124', 'SUL_T_81', 'SUL_T_4515', 'SUL_T_4235', 'SUL_T_4815',
       'SUL_T_4857', 'SUL_T_3998', 'SUL_T_4462', 'SUL_T_4108',
       'SUL_T_4174', 'SUL_T_3860', 'SUL_T_4162', 'SUL_T_4210',
       'SUL_T_233', 'SUL_T_1976', 'SUL_T_2017', 'SUL_T_308', 'SUL_T_1952',
       'SUL_T_777', 'SUL_T_1619', 'SUL_T_290', 'SUL_T_551', 'SUL_T_257',
       'SUL_T_343', 'SUL_T_362'], dtype=object)

In [47]:
v2_tables["SAMPLE"]['subject_id'].unique()

array(['SUL_T_362', 'SUL_T_308', 'SUL_T_3860', 'SUL_T_290', 'SUL_T_233',
       'SUL_T_4108', 'SUL_T_257', 'SUL_T_777', 'SUL_T_1976', 'SUL_T_4162',
       'SUL_T_4174', 'SUL_T_343', 'SUL_T_551', 'SUL_T_1619', 'SUL_T_1952',
       'SUL_T_2017', 'SUL_T_2465', 'SUL_T_4210', 'SUL_T_81', 'SUL_T_124',
       'SUL_T_2007', 'SUL_T_2791', 'SUL_T_3799', 'SUL_T_3870',
       'SUL_T_3998', 'SUL_T_4116', 'SUL_T_4235', 'SUL_T_4424',
       'SUL_T_4462', 'SUL_T_4515', 'SUL_T_4530', 'SUL_T_4695',
       'SUL_T_4815', 'SUL_T_4839', 'SUL_T_4857', 'SUL_T_4934',
       'SUL_T_4938', 'SUL_T_5133', 'SUL_T_5227', 'SUL_T_5259',
       'SUL_T_5359', 'SUL_T_5385', 'SUL_T_5498', 'SUL_T_5547'],
      dtype=object)

## v2->v3

In [48]:
v3_meta_tables = ['STUDY', 'PROTOCOL', 'SUBJECT', 'SAMPLE', 'DATA', 'CLINPATH', 'PMDBS', 'CONDITION', 'ASSAY_RNAseq']

f"{v3_meta_tables}"

"['STUDY', 'PROTOCOL', 'SUBJECT', 'SAMPLE', 'DATA', 'CLINPATH', 'PMDBS', 'CONDITION', 'ASSAY_RNAseq']"

In [49]:
v3_path = metadata_path / "v3"

v3_tables, aux_tables = v2_to_v3_PMDBS(v2_path, v3_path, CDEv2, CDEv3)

recoding number_of_brain_samples as int
recoding age_at_onset as int
recoding age_at_diagnosis as int
recoding first_motor_symptom as int
recoding replicate_count as int
recoding repeated_sample as int
recoding input_cell_count as int
recoding replicate_count as int
recoding repeated_sample as int


### validate v3 tables


In [50]:
CDE = CDEv3
for table,df in v3_tables.items():
    schema = CDE[CDE['Table'] == table]

    report = ReportCollector(destination="NA")
    full_table, report = validate_table(df.copy(), table, schema, report)
    report.print_log()

recoding number_samples as int
All required fields are present in *STUDY* table.
🚨⚠️❗ **8 Fields with empty (NULL) values:**

	- ASAP_grant_id: 1/1 empty rows (REQUIRED)

	- other_funding_source: 1/1 empty rows (REQUIRED)

	- publication_DOI: 1/1 empty rows (REQUIRED)

	- publication_PMID: 1/1 empty rows (REQUIRED)

	- PI_ORCID: 1/1 empty rows (OPTIONAL)

	- PI_google_scholar_id: 1/1 empty rows (OPTIONAL)

	- preprocessing_references: 1/1 empty rows (OPTIONAL)

	- alternate_dataset_id: 1/1 empty rows (OPTIONAL)
No invalid entries found in Enum fields.

All required fields are present in *PROTOCOL* table.
🚨⚠️❗ **3 Fields with empty (NULL) values:**

	- github_url: 1/1 empty rows (REQUIRED)

	- protocols_io_DOI: 1/1 empty rows (REQUIRED)

	- other_reference: 1/1 empty rows (OPTIONAL)
No invalid entries found in Enum fields.

All required fields are present in *SUBJECT* table.
🚨⚠️❗ **2 Fields with empty (NULL) values:**

	- race: 28/46 empty rows (REQUIRED)

	- primary_diagnosis_text: 46/

In [52]:
STUDY = v3_tables['STUDY']
STUDY

,ASAP_team_name,ASAP_lab_name,project_name,team_dataset_id,project_dataset,project_description,PI_full_name,PI_email,contributor_names,submitter_name,...,number_samples,sample_types,types_of_samples,DUA_version,metadata_tables,PI_ORCID,PI_google_scholar_id,preprocessing_references,metadata_version_date,alternate_dataset_id
0,TEAM-SULZER,Al-Dalahmah,The landscape of T-cell Immunity in Parkinson ...,sn_rnaseq,OJK_2022_ASAP_SEQ,Assessing the role of T cells in Parkinson's d...,Osama Al-Dalahmah,oa2298@cumc.columbia.edu,"Kelly A Jakubiak, Adithya Kannan, Jaeseung Lee...",Kelly Jakubiak,...,44,Cingulate Cortex and Substantia nigra,Postmortem PD and control,v1,"['STUDY', 'PROTOCOL', 'SUBJECT', 'SAMPLE', 'DA...",NA,NA,NA,v3.0_20241028,NA


-------------------------
## check md5s



In [6]:
print(team)

source = "pmdbs"

bucket = f"asap-raw-team-{team}-{source}-{dataset_name}"


key_file_path = Path.home() / f"Projects/ASAP/{team}-credentials.json"

res = authenticate_with_service_account(key_file_path)
print(res)

# make sure to get ALL the fastq files in the bucket
prefix = "**/*.gz"
bucket_files_md5 = get_md5_hashes( bucket, prefix)

sulzer
CompletedProcess(args='gcloud auth activate-service-account --key-file=/Users/ergonyc/Projects/ASAP/sulzer-credentials.json', returncode=0, stdout='', stderr='Activated service account credentials for: [raw-admin-sulzer@dnastack-asap-parkinsons.iam.gserviceaccount.com]\n')
gsutil -u dnastack-asap-parkinsons hash -h gs://asap-raw-team-sulzer-pmdbs-sn-rnaseq/**/*.gz


In [8]:
# def check_md5_sums()

v3_path = metadata_path / "v3"
data_df = read_meta_table(v3_path / "DATA.csv")

# checksum = v3_tables['DATA'][['file_name','file_MD5','file_type']].copy()

checksum = data_df[['file_name','file_MD5','file_type']].copy()
checksum = checksum[checksum.file_type == "fastq"]
checksum['check2'] = checksum['file_name'].map(bucket_files_md5)
checksum['check1'] = checksum['file_MD5']
checksum[checksum.check1 != checksum.check2].file_name.to_list()
#empty means success!!

[]

_____

## prep metadata in raw data bucket

steps:
- 1. archive whats there.  i.e. move to metadata/upload
- 2. copy metadata/upload to dataset upload (upload subdir)

In [10]:
metadata_subdir = "metadata"
current_files = gsutil_ls(bucket,metadata_subdir)

gsutil -u dnastack-asap-parkinsons ls gs://asap-raw-team-sulzer-pmdbs-sn-rnaseq/metadata
gsutil command succeeded: gs://asap-raw-team-sulzer-pmdbs-sn-rnaseq/metadata/CLINPATH.csv
gs://asap-raw-team-sulzer-pmdbs-sn-rnaseq/metadata/DATA.csv
gs://asap-raw-team-sulzer-pmdbs-sn-rnaseq/metadata/PROTOCOL.csv
gs://asap-raw-team-sulzer-pmdbs-sn-rnaseq/metadata/SAMPLE.csv
gs://asap-raw-team-sulzer-pmdbs-sn-rnaseq/metadata/STUDY.csv
gs://asap-raw-team-sulzer-pmdbs-sn-rnaseq/metadata/SUBJECT.csv
gs://asap-raw-team-sulzer-pmdbs-sn-rnaseq/metadata/v2-processed/
gs://asap-raw-team-sulzer-pmdbs-sn-rnaseq/metadata/v2/



In [11]:
metadata_subdir2 = "metadata/upload"
bucket = current_files[0].split("/")[2]

for file in current_files:
    if file == "":
        continue
    file_nm = Path(file).name
    is_dir = not file_nm.__contains__(".")

    source = f"gs://{bucket}/{metadata_subdir}/{file_nm}"

    destination = f"gs://{bucket}/{metadata_subdir2}/{file_nm}"
    gsutil_mv(source, destination, is_dir)

gsutil -u dnastack-asap-parkinsons mv gs://asap-raw-team-sulzer-pmdbs-sn-rnaseq/metadata/CLINPATH.csv gs://asap-raw-team-sulzer-pmdbs-sn-rnaseq/metadata/upload/CLINPATH.csv
gsutil command succeeded: 
gsutil -u dnastack-asap-parkinsons mv gs://asap-raw-team-sulzer-pmdbs-sn-rnaseq/metadata/DATA.csv gs://asap-raw-team-sulzer-pmdbs-sn-rnaseq/metadata/upload/DATA.csv
gsutil command succeeded: 
gsutil -u dnastack-asap-parkinsons mv gs://asap-raw-team-sulzer-pmdbs-sn-rnaseq/metadata/PROTOCOL.csv gs://asap-raw-team-sulzer-pmdbs-sn-rnaseq/metadata/upload/PROTOCOL.csv
gsutil command succeeded: 
gsutil -u dnastack-asap-parkinsons mv gs://asap-raw-team-sulzer-pmdbs-sn-rnaseq/metadata/SAMPLE.csv gs://asap-raw-team-sulzer-pmdbs-sn-rnaseq/metadata/upload/SAMPLE.csv
gsutil command succeeded: 
gsutil -u dnastack-asap-parkinsons mv gs://asap-raw-team-sulzer-pmdbs-sn-rnaseq/metadata/STUDY.csv gs://asap-raw-team-sulzer-pmdbs-sn-rnaseq/metadata/upload/STUDY.csv
gsutil command succeeded: 
gsutil -u dnastack

In [12]:
# verify
metadata_subdir = "metadata/upload"
current_files = gsutil_ls(bucket,metadata_subdir)

gsutil -u dnastack-asap-parkinsons ls gs://asap-raw-team-sulzer-pmdbs-sn-rnaseq/metadata/upload
gsutil command succeeded: gs://asap-raw-team-sulzer-pmdbs-sn-rnaseq/metadata/upload/CLINPATH.csv
gs://asap-raw-team-sulzer-pmdbs-sn-rnaseq/metadata/upload/DATA.csv
gs://asap-raw-team-sulzer-pmdbs-sn-rnaseq/metadata/upload/PROTOCOL.csv
gs://asap-raw-team-sulzer-pmdbs-sn-rnaseq/metadata/upload/SAMPLE.csv
gs://asap-raw-team-sulzer-pmdbs-sn-rnaseq/metadata/upload/STUDY.csv
gs://asap-raw-team-sulzer-pmdbs-sn-rnaseq/metadata/upload/SUBJECT.csv
gs://asap-raw-team-sulzer-pmdbs-sn-rnaseq/metadata/upload/v2-processed/
gs://asap-raw-team-sulzer-pmdbs-sn-rnaseq/metadata/upload/v2/



Archive the uploaded metadata locally



In [13]:
file_source = f"gs://{bucket}/{metadata_subdir}"
destination = f"{metadata_path}"
file_source, destination

('gs://asap-raw-team-sulzer-pmdbs-sn-rnaseq/metadata/upload',
 '/Users/ergonyc/Projects/ASAP/data/teams/sulzer/sn-rnaseq/metadata')

In [14]:

is_dir = True
gsutil_cp(file_source, destination, is_dir)

gsutil -u dnastack-asap-parkinsons cp -r gs://asap-raw-team-sulzer-pmdbs-sn-rnaseq/metadata/upload /Users/ergonyc/Projects/ASAP/data/teams/sulzer/sn-rnaseq/metadata
gsutil command succeeded: 


''

--------------------
## Create metadata package

This will copy the final updated to v3.0 metadata to `asap-could-processing-resources`


In [15]:
metadata_source = metadata_path


source = "pmdbs"
archive_root = Path.home() / "Projects/ASAP/asap-crn-metadata/datasets"
dataset_path = archive_root / f"{team}-{source}-{dataset_name}"
# bucket = f"asap-raw-data-team-{team}" # for now old locations
metadata_source, dataset_path

(PosixPath('/Users/ergonyc/Projects/ASAP/data/teams/sulzer/sn-rnaseq/metadata'),
 PosixPath('/Users/ergonyc/Projects/ASAP/asap-crn-metadata/datasets/sulzer-pmdbs-sn-rnaseq'))

________

In [16]:
fnms = create_metadata_package(metadata_source, dataset_path)


Copied /Users/ergonyc/Projects/ASAP/data/teams/sulzer/sn-rnaseq/metadata/v2 to /Users/ergonyc/Projects/ASAP/asap-crn-metadata/datasets/sulzer-pmdbs-sn-rnaseq/metadata/v2
Copied /Users/ergonyc/Projects/ASAP/data/teams/sulzer/sn-rnaseq/metadata/v3 to /Users/ergonyc/Projects/ASAP/asap-crn-metadata/datasets/sulzer-pmdbs-sn-rnaseq/metadata/v3
Copied /Users/ergonyc/Projects/ASAP/data/teams/sulzer/sn-rnaseq/metadata/og to /Users/ergonyc/Projects/ASAP/asap-crn-metadata/datasets/sulzer-pmdbs-sn-rnaseq/metadata/og
Copied /Users/ergonyc/Projects/ASAP/data/teams/sulzer/sn-rnaseq/metadata/upload to /Users/ergonyc/Projects/ASAP/asap-crn-metadata/datasets/sulzer-pmdbs-sn-rnaseq/metadata/upload


_____

generate ASAP IDs + transfering back to raw data bucket via `asap-crn-metadata` 

_____

## transfer metadata to raw data bucket

steps:
- 1. archive whats there.  i.e. move to metadata/archive
- 2. copy package to metadata/ . i.e. /og/*.csv, /v??/*.csv